In [1]:
from functools import partial

In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff

In [3]:
from ocddetection import data, federated
from ocddetection.federated.models import bidirectional

In [4]:
BATCH_SIZE = 5
WINDOW_SIZE = 10
FEATURE_SIZE = 3

In [5]:
def client_optimizer_fn():
    return tf.keras.optimizers.Adam(0.1)

def server_optimizer_fn():
    return tf.keras.optimizers.SGD(1.0)

In [6]:
def model_fn():
    return bidirectional.Bidirectional(
        WINDOW_SIZE,
        FEATURE_SIZE,
        10,
        16,
        0.4
    )

In [7]:
ds = tf.data.Dataset.from_tensor_slices(
    (
        np.random.random((12, WINDOW_SIZE, FEATURE_SIZE)),
        np.random.randint(0, 10, (12, WINDOW_SIZE))
    )
).batch(BATCH_SIZE)

In [8]:
fed_ds = [ds]

In [9]:
@tff.tf_computation
def server_init_tf():
    model = model_fn()
    server_optimizer = server_optimizer_fn()
    federated.__initialize_optimizer(model, server_optimizer)
    
    return federated.server.State(
        trainable_variables=model.trainable_variables,
        optimizer_state=server_optimizer.variables(),
        round_num=0
    )

In [10]:
@tff.federated_computation
def server_init_tff():
    """Orchestration logic for server model initialization."""
    return tff.federated_value(server_init_tf(), tff.SERVER)

In [12]:
server_init_tff()

RuntimeError: Cannot run the event loop while another loop is running